In [1]:
!pip install geemap
!pip install earthengine-api --upgrade
!pip install json
!pip install os
!pip install requests

import geemap
import ee
import json
import os
import requests

from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON, Marker, MarkerCluster

ERROR: Could not find a version that satisfies the requirement json
ERROR: No matching distribution found for json
ERROR: Could not find a version that satisfies the requirement os
ERROR: No matching distribution found for os


In [3]:
m01 = geemap.Map(center=[41.1044, 29.0281], zoom=15)
m01.add_basemap("TERRAIN")
m01

Map(center=[41.1044, 29.0281], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…

In [4]:
m_all = geemap.Map(center=[41.1044, 29.0281], zoom=15)
m_all.basemap_demo()
m_all

Map(center=[41.1044, 29.0281], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…

In [5]:
#GEE3 inspector tool
m_03 = geemap.Map(center=[41.1044, 29.0281], zoom=15)
dem = ee.Image('USGS/SRTMGL1_003')
landcover = ee.Image("ESA/GLOBCOVER_L4_200901_200912_V2_3").select('landcover')
landsat7 = ee.Image('LE7_TOA_5YEAR/1999_2003').select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7'])

vis_params = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Add Earth Eninge layers to Map
m_03.addLayer(dem, vis_params, 'SRTM DEM', True, 0.5)
m_03.addLayer(landcover, {}, 'Land cover')
m_03.addLayer(landsat7, {'bands': ['B4', 'B3', 'B2'], 'min': 20, 'max': 200}, 'Landsat 7')
m_03

Map(center=[41.1044, 29.0281], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…

In [28]:
basemaps = geemap.ee_basemaps.keys()
print(basemaps)

dict_keys(['ROADMAP', 'SATELLITE', 'TERRAIN', 'HYBRID', 'ESRI', 'Esri Ocean', 'Esri Satellite', 'Esri Standard', 'Esri Terrain', 'Esri Transportation', 'Esri Topo World', 'Esri National Geographic', 'Esri Shaded Relief', 'Esri Physical Map', 'FWS NWI Wetlands', 'FWS NWI Wetlands Raster', 'Google Maps', 'Google Satellite', 'Google Terrain', 'Google Satellite Hybrid', 'NLCD 2016 CONUS Land Cover', 'NLCD 2013 CONUS Land Cover', 'NLCD 2011 CONUS Land Cover', 'NLCD 2008 CONUS Land Cover', 'NLCD 2006 CONUS Land Cover', 'NLCD 2004 CONUS Land Cover', 'NLCD 2001 CONUS Land Cover', 'USGS NAIP Imagery', 'USGS Hydrography', 'USGS 3DEP Elevation', 'OpenStreetMap.Mapnik', 'OpenStreetMap.BlackAndWhite', 'OpenStreetMap.DE', 'OpenStreetMap.France', 'OpenStreetMap.HOT', 'Gaode.Normal', 'Gaode.Satellite', 'OpenTopoMap', 'Hydda.Full', 'Hydda.Base', 'Esri.WorldStreetMap', 'Esri.DeLorme', 'Esri.WorldTopoMap', 'Esri.WorldImagery', 'Esri.NatGeoWorldMap', 'HikeBike.HikeBike', 'MtbMap', 'CartoDB.Positron', 'Car

In [33]:
m_04 = geemap.Map(center=[41.1044, 29.0281], zoom=15)
m_04.split_map(left_layer='HYBRID', right_layer='TERRAIN')
m_04

Map(center=[41.1044, 29.0281], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…

In [34]:
#find Turkey's land Cover data
m_041 = geemap.Map()
m_041.split_map(left_layer='NLCD 2016 CONUS Land Cover', right_layer='NLCD 2001 CONUS Land Cover')
m_041

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [37]:
#find Turkey's land Cover data
collection = ee.ImageCollection("USGS/NLCD")
print(collection.aggregate_array('system:id').getInfo())

['USGS/NLCD/NLCD1992', 'USGS/NLCD/NLCD2001', 'USGS/NLCD/NLCD2001_AK', 'USGS/NLCD/NLCD2001_HI', 'USGS/NLCD/NLCD2001_PR', 'USGS/NLCD/NLCD2004', 'USGS/NLCD/NLCD2006', 'USGS/NLCD/NLCD2008', 'USGS/NLCD/NLCD2011', 'USGS/NLCD/NLCD2011_AK', 'USGS/NLCD/NLCD2011_HI', 'USGS/NLCD/NLCD2011_PR', 'USGS/NLCD/NLCD2013', 'USGS/NLCD/NLCD2016']


In [39]:
#find Turkey's land Cover data
nlcd_2001 = ee.Image('USGS/NLCD/NLCD2001').select('landcover')
nlcd_2016 = ee.Image('USGS/NLCD/NLCD2016').select('landcover')

left_layer = geemap.ee_tile_layer(nlcd_2001, {}, 'NLCD 2001')
right_layer = geemap.ee_tile_layer(nlcd_2016, {}, 'NLCD 2016')

m_042 = geemap.Map()
m_042.split_map(left_layer, right_layer)
m_042

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [42]:
#find Turkey's land Cover and boundry data
m_05 = geemap.Map()
image = ee.Image('USGS/SRTMGL1_003')

# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Add Earth Engine DEM to map
m_05.addLayer(image, vis_params, 'SRTM DEM')

states = ee.FeatureCollection("TIGER/2018/States")
m_05.addLayer(states, {}, 'US States')
m_05

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [45]:
m_051 = m_05

roi = ee.FeatureCollection(m_05.draw_features)
selected_states = states.filterBounds(roi)
m_051.addLayer(selected_states, {}, "Selected states")

clipped_image = image.clip(selected_states)
m_051.addLayer(clipped_image, vis_params, 'Clipped image')
m_051

Map(bottom=1079.0, center=[39.50404070558415, -66.09375000000001], controls=(WidgetControl(options=['position'…

In [65]:
m_06 = geemap.Map()

path_06 = r"D:\Desktop\rs\data\us-cities.json"
file_path_06 = os.path.abspath(path_06)

with open(file_path_06) as f:
    json_data_06 = json.load(f)

maker_cluster_06 = MarkerCluster(
markers_06=[Marker(location=feature['geometry']['coordinates'][::-1]) for feature in json_data_06['features']],
name = 'Markers')

m_06.add_layer(maker_cluster_06)
ee_fc = geojson_to_ee(json_data_06)

m_06.addLayer(ee_fc, {}, "US Cities EE")

m_06

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [69]:
m_071 = geemap.Map()
m_072 = geemap.Map()
m_073 = geemap.Map()

path_07 = r"D:\Desktop\rs\data\us-states.json"
file_path_07 = os.path.abspath(path_07)

with open(file_path_07) as f:
    json_data_07 = json.load(f)

json_layer_07 = GeoJSON(data=json_data_07, name='US States JSON', hover_style={'fillColor': 'red' , 'fillOpacity': 0.5})
m_071.add_layer(json_layer_07)

m_071

In [71]:
ee_data_07 = geojson_to_ee(json_data_07)
m_072.addLayer(ee_data_07, {}, "US States EE")

m_072

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [72]:
json_data_071 = ee_to_geojson(ee_data_07)
json_layer_071 = GeoJSON(data=json_data_071, name='US States EE JSON', hover_style={'fillColor': 'red' , 'fillOpacity': 0.5})
m_073.add_layer(json_layer_071)

m_073

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [73]:
m_074 = geemap.Map()

path_071 = r"D:\Desktop\rs\data\countries.json"
file_path_071 = os.path.abspath(path_07)

with open(file_path) as f:
    json_data_072 = json.load(f)
    
json_layer_072 = GeoJSON(data=json_data_072, name='Counties', hover_style={'fillColor': 'red' , 'fillOpacity': 0.5})
m_074.add_layer(json_layer_072)

m_074

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…